In [7]:
from openai import OpenAI
from random import randint, choice
from os import environ
from pathlib import Path
from json import loads, dumps
environ["OPENAI_API_KEY"] = Path("~/.openaiapikey").expanduser().read_text().strip()

openaiClient = OpenAI()
def gpt_3_5_turbo_completion(query):
    answer = openaiClient.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def gpt_4_turbo_completion(query):
    answer = openaiClient.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def tryRecieveAnswer(query, completionFunction = gpt_3_5_turbo_completion, answerConversion = lambda x: x, maxTries = 10):
    tryNumber = 0
    while tryNumber < maxTries:
        answer = completionFunction(query)
        try:
            answer = answerConversion(answer)
            return (answer, True)
        except:
            pass
        tryNumber += 1
    print(f"Failed to recieve answer for query: {query}. Last answer: {answer}")
    return (None, False)

def listAnswerConversion(answer):
    result = loads(answer)
    assert isinstance(result, list)
    for item in result:
        assert isinstance(item, str)
    return result

In [2]:
gpt_4_turbo_completion('''
I want to build an ontology. I am using the following owl classes: {1 : "Scientist", 2 : "Material", 3 : "Particle", 4 : "Force"}. 
What relations could exist between instances of these classes?
Return a list of possible relations in the following format: 
[[<number of the subject class>, "name of the first relation", <number of the object class>],
[<number of the subject class>, "name of the second relation", <number of the object class>], ... ]
Return nothing but this list.''')

'[[1, "researches", 2],\n [1, "studies", 3],\n [1, "investigates", 4],\n [2, "composed_of", 3],\n [3, "subject_to", 4]]'

In [3]:
gpt_4_turbo_completion('''
I want to build an ontology about physics. 
Therefor I need to create a list of owl classes. I already have the owl classes {"Scientist", "Material", "Particle", "Force"}. 
What could be 5 additional owl classes that I could use in my ontology?
Return a list of 5 owl classes formatted as follows: ["name of the first class", "name of the second class", ...]
Write the names in camel case and return nothing but this list.''')

'["Energy", "Experiment", "Theory", "Equation", "Field"]'

In [1]:
from pathlib import Path
datapath = Path("../master-database-files/master-experimental/generated_ontology/")
assert datapath.exists()

In [5]:
from json import loads, dumps

In [17]:
def addOntologyClasses(nameOfOntology, ontologyTopic = "physics", numberOfEntries = 5, seed = []):
    ontologyPath = datapath / nameOfOntology
    ontologyPath.mkdir(exist_ok = True)
    classesPath = ontologyPath / "classes.json"
    if not classesPath.exists():
        classesPath.write_text(dumps([]))
    classes = loads(classesPath.read_text())
    if classes == []:
        classes = seed
    query = f'''
I want to build an ontology about {ontologyTopic}.
Therefor I need to create a list of owl classes. I already have the owl classes {dumps(classes)}.
What could be {numberOfEntries} additional owl classes that I could use in my ontology?
Return a list of {numberOfEntries} owl classes formatted as follows: ["name of the first class", "name of the second class", ...]
Write the names in camel case and return nothing but this list.'''
    def answerConversion(answer):
        result = loads(answer)
        assert isinstance(result, list)
        assert len(result) == numberOfEntries
        for item in result:
            assert isinstance(item, str)
        return result
    answer, success = tryRecieveAnswer(query, gpt_4_turbo_completion, answerConversion=answerConversion)
    if success:
        classes += answer
        classesPath.write_text(dumps(classes, indent=4))
    else:
        print(f"Failed to add classes to {classesPath}")

In [51]:
def addOntologyRelations(nameOfOntology):
    ontologyPath = datapath / nameOfOntology
    ontologyPath.mkdir(exist_ok = True)
    classesPath = ontologyPath / "classes.json"
    relationsPath = ontologyPath / "relations.json"
    if not relationsPath.exists():
        relationsPath.write_text(dumps([]))
    classes = loads(classesPath.read_text())
    relations = loads(relationsPath.read_text())
    numberedClassesString = "{" + ", ".join([f"{i+1} : \"{classes[i]}\"" for i in range(len(classes))]) + "}"
    query = f'''
I want to build an ontology. I am using the following owl classes: {numberedClassesString}.
What relations could exist between instances of these classes?
Return a list of possible relations in the following format: 
[[<number of the subject class>, "name of the first relation", <number of the object class>],
[<number of the subject class>, "name of the second relation", <number of the object class>], ... ]
The relation names should be in camel case.
Return nothing but this list.'''
    def answerConversion(answer):
        result = loads(answer)
        assert isinstance(result, list)
        for item in result:
            assert isinstance(item, list)
            assert len(item) == 3
            assert isinstance(item[0], int)
            assert item[0] >= 1 and item[0] <= len(classes)
            assert isinstance(item[1], str)
            assert isinstance(item[2], int)
            assert item[2] >= 1 and item[2] <= len(classes)
        return result
    answer, success = tryRecieveAnswer(query, gpt_4_turbo_completion, answerConversion=answerConversion)
    if success:
        for item in answer:
            # Check if there is already a relation that links the same classes
            if not any([(relation[0] == classes[item[0]-1] and relation[2] == classes[item[2]-1] ) or (relation[0] == classes[item[2]-1] and relation[2] == classes[item[0]-1]) for relation in relations]):
                relations.append([classes[item[0]-1], item[1], classes[item[2]-1]])
        relationsPath.write_text(dumps(relations, indent=4))
    else:
        print(f"Failed to add relations to {relationsPath}")

In [46]:
from rdflib import Graph, URIRef, Literal, BNode, Namespace
def convertOntologyToRDF(nameOfOntology, baseURI = "http://quantsimulant.de/owl/generated_ontology/"):
    ontologyPath = datapath / nameOfOntology
    classesPath = ontologyPath / "classes.json"
    relationsPath = ontologyPath / "relations.json"
    classes = loads(classesPath.read_text())
    relations = loads(relationsPath.read_text())
    ontology = Graph()
    onto = Namespace(baseURI)
    ontology.bind("onto", onto)
    for className in classes:
        ontology.add((onto[className], URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), URIRef("http://www.w3.org/2002/07/owl#Class")))
    relationNamesCounter = {}
    for relation in relations:
        counter = relationNamesCounter.get(relation[1], 0)
        relationNamesCounter[relation[1]] = counter + 1
        if counter > 0:
            relationName = f"{relation[1]}_{counter + 1}"
        else:
            relationName = relation[1]
        ontology.add((onto[relationName], URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), URIRef("http://www.w3.org/2002/07/owl#ObjectProperty")))
        ontology.add((onto[relationName], URIRef("http://www.w3.org/2000/01/rdf-schema#domain"), onto[relation[0]]))
        ontology.add((onto[relationName], URIRef("http://www.w3.org/2000/01/rdf-schema#range"), onto[relation[2]]))
    ontology.serialize(str(ontologyPath / "ontology.ttl"), format="turtle")


In [18]:
addOntologyClasses("genPhysOnto", "physics", 5, seed = ["Scientist", "Publication", "Experiment"])

In [54]:
addOntologyClasses("unseededOnto", "physics", 5)

In [24]:
addOntologyClasses("researchAreaSeeded", "physics", 5, seed = ["ResearchArea"])

In [52]:
addOntologyRelations("researchAreaSeeded")

In [53]:
convertOntologyToRDF("researchAreaSeeded")